<a href="https://colab.research.google.com/github/kuo-chuan/Python/blob/main/%E5%8F%A3%E7%BD%A9%E8%B3%87%E6%96%99_API_%E4%B8%B2%E6%8E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

口罩實名制在去年初開始實施，為了避免像之前那樣出現排隊搶購的情況，健保署規劃提供即時的口罩庫存資訊給民眾參考，此口罩地圖 API 網址合併了多種開放資料，包含著特約藥局、診所相關資訊，以及健保署提供的口罩剩餘數量明細。

In [1]:
import requests
import json
import pandas as pd

In [2]:
# 收集
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
r = requests.get(url)
response = r.text
# 整理
data = json.loads(response)
with open("mask.json", "w") as f:
    f.write(json.dumps(data))
with open('mask.json', 'r') as f:
    data = json.load(f)

In [3]:
#計算各地區的藥局數量
from collections import defaultdict
med_count = defaultdict(int)
for row in data['features']:
    conunty = row['properties']['county']
    med_count[conunty] += 1

print(med_count)

defaultdict(<class 'int'>, {'臺北市': 339, '': 49, '高雄市': 422, '臺中市': 420, '臺南市': 271, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 501, '桃園市': 264, '新竹縣': 46, '宜蘭縣': 76, '苗栗縣': 56, '彰化縣': 179, '南投縣': 67, '雲林縣': 129, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 46, '臺東縣': 23, '金門縣': 6, '連江縣': 1})


In [4]:
# 試著計算每個地區的剩餘口罩數量(分成人的和小孩的)，並且從大到小排列
# 宣告 dictionary 用來存放資料
child_count = defaultdict(int)
adult_count = defaultdict(int) 
 
# 將資料迭代印出做計算
for row in data['features']:
    conunty = row['properties']['county']
    mask_child = row['properties']['mask_child']
    mask_adult = row['properties']['mask_adult']
    child_count[conunty] += mask_child
    adult_count[conunty] += mask_adult

# 排序
print(sorted(adult_count.items(), key=lambda kv: kv[1], reverse=True))
print(sorted(child_count.items(), key=lambda kv: kv[1], reverse=True))

[('新北市', 670270), ('臺中市', 536710), ('高雄市', 424170), ('臺南市', 393900), ('桃園市', 331940), ('臺北市', 322490), ('彰化縣', 219400), ('雲林縣', 196990), ('屏東縣', 184250), ('苗栗縣', 131080), ('嘉義縣', 113180), ('嘉義市', 107840), ('宜蘭縣', 106900), ('南投縣', 91240), ('基隆市', 80100), ('', 69210), ('花蓮縣', 57330), ('新竹縣', 51350), ('新竹市', 45630), ('臺東縣', 28510), ('金門縣', 15510), ('澎湖縣', 12600), ('連江縣', 7930)]
[('新北市', 539260), ('臺中市', 470910), ('高雄市', 421180), ('臺北市', 344150), ('臺南市', 327950), ('桃園市', 270810), ('彰化縣', 220060), ('屏東縣', 169820), ('雲林縣', 165570), ('嘉義縣', 112370), ('嘉義市', 97820), ('宜蘭縣', 93670), ('南投縣', 91630), ('基隆市', 84380), ('苗栗縣', 82630), ('花蓮縣', 57190), ('', 56130), ('新竹縣', 55630), ('新竹市', 43580), ('臺東縣', 25810), ('澎湖縣', 16510), ('金門縣', 14980), ('連江縣', 3470)]


In [5]:
# 透過 Pandas 套件計算
properties = []

for info in data['features']:
    properties += [info['properties']]

properties = pd.DataFrame(properties)

total = properties[["county", "mask_adult", "mask_child"]]
total_sum = (total.groupby(by=['county']).sum()).sort_values(by=["mask_adult", "mask_child"], ascending=False)
print(total_sum)

        mask_adult  mask_child
county                        
新北市         670270      539260
臺中市         536710      470910
高雄市         424170      421180
臺南市         393900      327950
桃園市         331940      270810
臺北市         322490      344150
彰化縣         219400      220060
雲林縣         196990      165570
屏東縣         184250      169820
苗栗縣         131080       82630
嘉義縣         113180      112370
嘉義市         107840       97820
宜蘭縣         106900       93670
南投縣          91240       91630
基隆市          80100       84380
             69210       56130
花蓮縣          57330       57190
新竹縣          51350       55630
新竹市          45630       43580
臺東縣          28510       25810
金門縣          15510       14980
澎湖縣          12600       16510
連江縣           7930        3470
